In [4]:
# Dashbords Datasets
Sales_full = 'Sales_full'
Sales_mini = 'Sales_mini'
Sales_mini = 'Sales_mini'
FBA_Fees='FBA Fees'
Current_price = 'Current_price'

In [5]:
from os import listdir
from os.path import isfile, join
from openpyxl import load_workbook
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from scipy.stats import ttest_ind
from google.colab import drive

In [6]:
drive.mount('/content/drive')
os.getcwd()

Mounted at /content/drive


'/content'

# My Functions

In [7]:
def group_d(ASIN, Start_Date, End_Date, Marketplace):
    df_p =pd.DataFrame()
# Filtering Based on the Marketplace and the Date
    df_p = df_sf[df_sf['Marketplace'] == Marketplace]
    df_p = df_p[df_p['Date'].between(Start_Date, End_Date)]
    df_p = df_p[df_p['ASIN'] == ASIN]

    try:


# Reset index for better indexing of the data
        df_p.reset_index(inplace = True)
# Aggregate the columns of Sales, sold units, and FBA fees columns based on ASINs
        df_p = pd.pivot_table(df_p,index=['ASIN'], values=(['SalesOrganic',
                                                            'SalesSponsoredProducts',
                                                            'SalesSponsoredDisplay',
                                                            'UnitsSponsoredDisplay',
                                                            'UnitsOrganic',
                                                            'UnitsPPC',
                                                            'UnitsSponsoredProducts',
                                                            'FBAStorageFee',
                                                            'FBALongTermStorageFee',
                                                           'FBAPerUnitFulfillmentFee']), aggfunc = 'sum')

# Calculate the Gross Revenue, Units Sold, and FBA storage fee columns
        df_p['Gross Revenue'] = df_p['SalesOrganic'] +  df_p['SalesSponsoredProducts'] + df_p['SalesSponsoredDisplay']
        df_p['Units Sold'] = df_p['UnitsOrganic'] + df_p['UnitsPPC']
        df_p['FBA storage fee'] = df_p['FBALongTermStorageFee'] + df_p['FBAStorageFee'] +  df_p['FBAPerUnitFulfillmentFee']

# Drop the useless Columns
        df_p.drop(columns= {'SalesOrganic',
                            'SalesSponsoredProducts',
                            'SalesSponsoredDisplay',
                            'UnitsSponsoredDisplay',
                            'UnitsOrganic',
                            'UnitsPPC',
                            'FBALongTermStorageFee',
                            'FBAStorageFee',
                            'UnitsSponsoredProducts',
                            'FBAPerUnitFulfillmentFee'}, inplace = True)
    except:
        pass
# Return the targeted dataset
    return df_p

In [8]:
def group_a(ASIN, Start_Date, End_Date, Marketplace, Price):
    try:
        df_p_m = df_sm[df_sm['Marketplace'] == Marketplace]
        df_p_m = df_p_m[df_p_m['Date'].between(Start_Date, End_Date)]
        df_p_m = pd.pivot_table(df_p_m,index=['ASIN', 'Marketplace'], values=(['RefundCost','NetProfit', 'Cost of Goods']), aggfunc = 'sum')
        df_p_m.reset_index(inplace = True)
        df_p_m['Price'] = Price
    except:
        pass
    return df_p_m

In [9]:
 def get_seller_board(ASIN,Start_Date, End_Date, Marketplace, Price, days):
    df_p = group_a(ASIN, Start_Date, End_Date, Marketplace, Price).merge(group_d(ASIN, Start_Date, End_Date, Marketplace), on= 'ASIN')
    df_p.merge(group_storage_fees(ASIN, Start_Date, End_Date, Marketplace), on='ASIN', how = 'left')
    df_p['Start_Date']=Start_Date
    df_p['End_Date']=End_Date
    df_p['No. of days']=days
    return df_p

In [10]:
def group_storage_fees(ASIN, Start_Date, End_Date, Marketplace):
    try:
        df_p_m = df_sf[df_sf['Marketplace'] == Marketplace]
        df_p_m = df_p_m[df_p_m['Date'].between(Start_Date, End_Date)]
        df_p_m = pd.pivot_table(df_p_m,index=['ASIN', 'Marketplace'], values=(['FBAStorageFee']), aggfunc = 'sum')
        df_p_m.reset_index(inplace = True)
    except:
        pass
    return df_p_m

In [11]:
def Final_FBA_Storage_Fee(ASIN, month_of_charge_start, month_of_charge_end , year_of_charge_start, year_of_charge_end, Region):
    df_f = df_f_t[df_f_t['Month'].between(month_of_charge_start, month_of_charge_end)]
    df_f = df_f[df_f['ASIN'] == ASIN]
    df_f = df_f[(df_f['Year'] == year_of_charge_start) | (df_f['Year'] == year_of_charge_end)]
    df_f = df_f[df_f['Region'] == Region]
    df_f = pd.pivot_table(df_f,index=['ASIN', 'Month'], values=(['Final FBA storage fee',
                                                                 'Final FBA storage fee Per day']), aggfunc = 'sum').reset_index()
    return df_f

In [12]:
def Overcome_null_values(df_t):
    df = pd.DataFrame()
    df_tt = pd.DataFrame()
    for i in df_t["ASIN"].unique():
        df_tt = df_t[df_t["ASIN"] == i]
        df_tt.reset_index(inplace = True)
        df_tt.drop(columns = 'index', inplace = True)
        index = df_tt[df_tt['Price'].isna()].index
        list_of_bad_index  = []
        for x in index:
            if x == 0 or x +1 == df_tt.shape[0]:
                list_of_bad_index.append(x)
            else:
                df_tt.loc[x-1,'No. of days'] = int(df_tt.loc[x-1,'No. of days']) + int(df_tt.loc[x+1,'No. of days'])
                df_tt.loc[x-1,'End_Date'] = df_tt.loc[x+1,'End_Date']
                list_of_bad_index.append(x+1)
                list_of_bad_index.append(x)
        df_tt.drop(index = list_of_bad_index, inplace = True)
        df = pd.concat([df,df_tt ])
        df.reset_index(inplace = True)
        df.drop(columns = 'index', inplace = True)
    return df

# Required Data

## Sales Data


In [13]:
root_path = r'/content/drive/My Drive/Sales Data/Final Shape of Files'
os.chdir(root_path)
df_sf = pd.read_csv(Sales_full)
df_sm = pd.read_csv(Sales_mini)
df_sf.fillna(0, inplace = True)
df_sm.fillna(0, inplace = True)

## Prodduct Current Prices Data

In [14]:
df_cp = pd.read_csv(Current_price)

# Data Aggregation

In [15]:
df_cp["End_Date"] = pd.to_datetime(df_cp["End_Date"], format='mixed').dt.date
df_sf['Date'] = pd.to_datetime(df_sf['Date'], format = "%d/%m/%Y")
df_sm['Date'] = pd.to_datetime(df_sm['Date'], format = "%d/%m/%Y")

In [16]:
df_cp['Start_Date'] = pd.to_datetime(df_cp['End_Date'])
df_cp['End_Date'] = pd.to_datetime(df_cp['Start_Date'])
df_cp.drop_duplicates(subset= {'Start_Date', 'ASIN', 'Marketplace'}, inplace = True)
df_cp = Overcome_null_values(df_cp)

/tmp/ipython-input-12-4267985935.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tt.drop(columns = 'index', inplace = True)
/tmp/ipython-input-12-4267985935.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tt.drop(index = list_of_bad_index, inplace = True)
/tmp/ipython-input-12-4267985935.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tt.drop(columns = 'index', inplace = True)
/tmp/ipython-input-12-4267985935.py:1

In [17]:
last_data_point = input('Enter last data date for the analysis like this "2025-03-31"')
df_cp_overdate = df_cp[df_cp['Date'] > last_data_point]
df_cp.drop(df_cp_overdate.index, inplace = True)
ind = df_cp[df_cp['End_Date'] > last_data_point].index
df_cp.loc[ind, 'End_Date'] = last_data_point
df_cp.reset_index(inplace = True, drop = True)
df_cp['No. of days'] = ( pd.to_datetime(df_cp['End_Date']) - pd.to_datetime(df_cp['Date'])).dt.days + 1

Enter last data date for the analysis like this "2025-03-31"2025-05-31


In [18]:
df_t = pd.DataFrame()
for i in range(0, df_cp.shape[0]):
    Start_Date =str(df_cp.loc[i, 'Date'])
    End_Date = str(df_cp.loc[i, 'End_Date'])
    Marketplace = df_cp.loc[i, 'Marketplace']
    Price = df_cp.loc[i, 'Price']
    ASIN = df_cp.loc[i, 'ASIN']
    days = df_cp.loc[i, 'No. of days']
    df_t = pd.concat([df_t,
    get_seller_board(ASIN,Start_Date, End_Date, Marketplace, Price, days )])
    if i == 0:
        print(df_cp.shape[0])
    if i % 100 == 0:
        print(i)

21317
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18

In [19]:
# Convert columns to datetime format
df_t['Start_Date'] = pd.to_datetime(df_t['Start_Date'])
df_t['End_Date'] = pd.to_datetime(df_t['End_Date'])

# Calculate the number of days
df_t['Days_Difference'] = (df_t['End_Date'] - df_t['Start_Date']).dt.days +1


In [20]:
df_t = df_t[['ASIN', 'Marketplace', 'Start_Date', 'End_Date',
             'No. of days' ,'Price', 'Gross Revenue', 'Units Sold', 'NetProfit', 'RefundCost', 'FBA storage fee', 'Cost of Goods']]
df_t.reset_index(inplace = True)
df_t.drop(columns = 'index',inplace = True)


In [21]:
df_c = df_t.copy()
df_c[ 'Average units sold per day'] = df_c['Units Sold'] / df_c['No. of days']
df_c[ 'Expenses'] = df_c[ 'Gross Revenue'] - df_c['NetProfit']
df_c['Net Margin'] = df_c['NetProfit'] / df_c['Gross Revenue']
df_c['Average Sales per day'] = df_c['Gross Revenue'] / df_c['No. of days']
df_c['Average Daily Profit'] = df_c['NetProfit'] / df_c['No. of days']
df_c['NetProfit Without COGS']= df_c['NetProfit'] + (df_c['Cost of Goods'] +df_c['FBA storage fee'] +df_c['RefundCost'])*-1
df_c['Average Daily Profit Without COGS'] = df_c['NetProfit Without COGS'] / df_c['No. of days']
df_c['Net Margin Without COGS'] =df_c['NetProfit Without COGS'] / df_c['Gross Revenue']

In [22]:
df_c['Start_Date'] = pd.to_datetime(df_c['Start_Date']).dt.date
df_c['End_Date'] = pd.to_datetime(df_c['End_Date']).dt.date

In [23]:
root_path = r"/content/drive/My Drive/Sales Data/Sales Analysis"
os.chdir(root_path)
df_tt = pd.DataFrame()
for i in df_t['Marketplace'].unique():
    df_tt = df_c[df_c['Marketplace'] == i]
    df_tt.reset_index(inplace = True)
    df_tt.drop(columns = 'index',inplace = True)
    df_tt.to_excel(i+'.xlsx', index = False)



df_cp_overdate.to_excel('overdate changes'+'.xlsx', index = False)